In [1]:
# импортируем необходимые библиотеки, классы и функции
import h2o
import pandas as pd
import numpy as np
from sklearn.model_selection import (train_test_split, 
                                     GridSearchCV)
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from category_encoders import CountEncoder
from h2o.sklearn import (H2OAutoMLClassifier, 
                         H2OAutoMLRegressor)
from sklearn.impute import SimpleImputer
from sklearn.metrics import (accuracy_score,
                             mean_squared_error)

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
# отключаем предупреждения
import warnings
warnings.filterwarnings(action='ignore')

In [3]:
# запускаем кластер H2O
h2o.init(nthreads=-1, max_mem_size=8)

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.15" 2022-04-19; OpenJDK Runtime Environment (build 11.0.15+10-Ubuntu-0ubuntu0.18.04.1); OpenJDK 64-Bit Server VM (build 11.0.15+10-Ubuntu-0ubuntu0.18.04.1, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.7/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpbci2cj48
  JVM stdout: /tmp/tmpbci2cj48/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpbci2cj48/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,04 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.36.1.2
H2O_cluster_version_age:,1 month and 3 days
H2O_cluster_name:,H2O_from_python_unknownUser_v67wdh
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,8 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


## `H2OAutoMLClassifier` пример

In [4]:
# загружаем данные для задачи классификации
data = pd.read_csv('Data/StateFarm_missing.csv', sep=';')
data.head()

,Customer Lifetime Value,Coverage,Education,EmploymentStatus,Gender,Income,Monthly Premium Auto,Months Since Last Claim,Months Since Policy Inception,Number of Open Complaints,Number of Policies,Response
0,2763.519279,Basic,Bachelor,Employed,F,56274.0,NaN,32.0,5.0,NaN,1.0,No
1,NaN,NaN,Bachelor,Unemployed,F,0.0,NaN,13.0,42.0,NaN,NaN,No
2,NaN,NaN,NaN,Employed,F,48767.0,108.0,NaN,38.0,0.0,NaN,No
3,7645.861827,Basic,Bachelor,NaN,NaN,0.0,106.0,18.0,NaN,NaN,7.0,No
4,2813.692575,Basic,Bachelor,NaN,M,43836.0,73.0,12.0,NaN,NaN,1.0,No


In [5]:
# разбиваем данные на обучающие и тестовые: получаем обучающий
# массив признаков, тестовый массив признаков, обучающий массив
# меток, тестовый массив меток
X_train, X_test, y_train, y_test = train_test_split(
    data.drop('Response', axis=1), 
    data['Response'], 
    test_size=0.3,
    stratify=data['Response'],
    random_state=42)

# превращаем датафреймы в массивы NumPy
X_train = X_train.values
X_test = X_test.values

y_train = y_train.values
y_test = y_test.values

# создаем конвейер
pipe = Pipeline([
    ('imp', SimpleImputer(strategy='constant')),
    ('classifier', H2OAutoMLClassifier(preprocessing=None, 
                                       max_models=10, seed=2022))
])

# запускаем AutoML
pipe.fit(X_train, y_train);

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%


In [6]:
# получаем прогнозы для тестовой выборки
predictions = pipe.predict(X_test)
# оценим правильность на тестовой выборке
acc = accuracy_score(y_test, predictions)
print("Правильность на тестовой выборке: %.3f" % acc)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%
Правильность на тестовой выборке: 0.941


In [7]:
# выводим результаты AutoML
automl = pipe.named_steps.classifier.estimator
automl.leaderboard

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
GBM_4_AutoML_1_20220629_182612,0.913029,0.17163,0.629657,0.0897083,0.216536,0.046888
StackedEnsemble_BestOfFamily_1_AutoML_1_20220629_182612,0.912744,0.169933,0.628493,0.0919027,0.217101,0.0471331
StackedEnsemble_AllModels_1_AutoML_1_20220629_182612,0.912073,0.169591,0.629174,0.0909478,0.216839,0.047019
GBM_3_AutoML_1_20220629_182612,0.910665,0.174784,0.630274,0.0927643,0.219404,0.0481382
GBM_2_AutoML_1_20220629_182612,0.909894,0.180035,0.618754,0.115197,0.222876,0.0496736
XGBoost_2_AutoML_1_20220629_182612,0.908942,0.192712,0.592121,0.164934,0.233423,0.0544862
GBM_1_AutoML_1_20220629_182612,0.906138,0.192051,0.592964,0.134694,0.233383,0.0544677
XGBoost_3_AutoML_1_20220629_182612,0.904502,0.193615,0.579166,0.117902,0.232535,0.0540726
XRT_1_AutoML_1_20220629_182612,0.901894,0.218027,0.624927,0.115005,0.226119,0.0511296
DRF_1_AutoML_1_20220629_182612,0.898166,0.30724,0.616114,0.10327,0.226227,0.0511785


In [8]:
# создаем конвейер
pipe = Pipeline([
    ('imp', SimpleImputer(strategy='constant')),
    ('classifier', H2OAutoMLClassifier(preprocessing=None, 
                                       exclude_algos=['DRF'],
                                       max_models=10, seed=2022))
])

# запускаем AutoML
pipe.fit(X_train, y_train);

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%


In [9]:
# получаем прогнозы для тестовой выборки
predictions = pipe.predict(X_test)
# оценим правильность на тестовой выборке
acc = accuracy_score(y_test, predictions)
print("Правильность на тестовой выборке: %.3f" % acc)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%
Правильность на тестовой выборке: 0.942


In [10]:
# выводим результаты AutoML
automl = pipe.named_steps.classifier.estimator
automl.leaderboard

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
GBM_3_AutoML_2_20220629_182857,0.91451,0.174031,0.628115,0.0897157,0.219013,0.0479669
StackedEnsemble_BestOfFamily_1_AutoML_2_20220629_182857,0.913688,0.173071,0.627176,0.0899072,0.219735,0.0482833
GBM_4_AutoML_2_20220629_182857,0.9135,0.171677,0.635448,0.0928575,0.216805,0.0470044
StackedEnsemble_AllModels_1_AutoML_2_20220629_182857,0.913416,0.169593,0.632087,0.0929533,0.216986,0.0470829
GBM_2_AutoML_2_20220629_182857,0.909623,0.177571,0.628567,0.0968757,0.221213,0.0489353
XGBoost_2_AutoML_2_20220629_182857,0.908942,0.192712,0.592121,0.164934,0.233423,0.0544862
GBM_5_AutoML_2_20220629_182857,0.907962,0.18445,0.615031,0.123695,0.225956,0.0510559
XGBoost_3_AutoML_2_20220629_182857,0.907085,0.189672,0.600344,0.141068,0.228614,0.0522642
GBM_1_AutoML_2_20220629_182857,0.906138,0.192051,0.592964,0.134694,0.233383,0.0544677
XGBoost_1_AutoML_2_20220629_182857,0.892651,0.208908,0.536815,0.185764,0.24532,0.0601819


In [11]:
# разбиваем данные на обучающие и тестовые: получаем обучающий
# массив признаков, тестовый массив признаков, обучающий массив
# меток, тестовый массив меток
X_train, X_test, y_train, y_test = train_test_split(
    data.drop('Response', axis=1), 
    data['Response'], 
    test_size=0.3,
    stratify=data['Response'],
    random_state=42)

# создаем списки категориальных 
# и количественных столбцов
cat_columns = X_train.select_dtypes(
    include='object').columns.tolist()
num_columns = X_train.select_dtypes(
    exclude='object').columns.tolist()

# массивы меток (серии) превращаем в массивы NumPy
y_train = y_train.values
y_test = y_test.values

# создаем конвейер для количественных переменных
num_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='median'))
])

# создаем конвейер для категориальных переменных
cat_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('count', CountEncoder(return_df=False))
])

# создаем список трехэлементных кортежей, в котором
# первый элемент кортежа - название конвейера с
# преобразованиями для определенного типа признаков
transformers = [('num', num_pipe, num_columns),
                ('cat', cat_pipe, cat_columns)]

# создаем конвейер
pipe = Pipeline([
    ('tr', ColumnTransformer(transformers=transformers)),
    ('classifier', H2OAutoMLClassifier(preprocessing=None,
                                       max_models=10, seed=2022))])

# запускаем AutoML
pipe.fit(X_train, y_train);

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%


In [12]:
# получаем прогнозы для тестовой выборки
predictions = pipe.predict(X_test)
# оценим правильность на тестовой выборке
acc = accuracy_score(y_test, predictions)
print("Правильность на тестовой выборке: %.3f" % acc)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%
Правильность на тестовой выборке: 0.938


In [13]:
# выводим результаты AutoML
automl = pipe.named_steps.classifier.estimator
automl.leaderboard

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
StackedEnsemble_BestOfFamily_1_AutoML_3_20220629_183108,0.913198,0.170277,0.628174,0.0936234,0.217896,0.0474785
StackedEnsemble_AllModels_1_AutoML_3_20220629_183108,0.912461,0.170575,0.625438,0.0912375,0.218012,0.0475291
GBM_4_AutoML_3_20220629_183108,0.912366,0.172513,0.627018,0.0901869,0.217613,0.0473553
XGBoost_2_AutoML_3_20220629_183108,0.910819,0.189462,0.601831,0.136795,0.231553,0.0536167
GBM_2_AutoML_3_20220629_183108,0.909529,0.179718,0.621232,0.0988836,0.222935,0.0496999
GBM_3_AutoML_3_20220629_183108,0.908902,0.176946,0.62083,0.0928649,0.220594,0.0486618
GBM_1_AutoML_3_20220629_183108,0.904372,0.195252,0.581743,0.141662,0.235526,0.0554724
DRF_1_AutoML_3_20220629_183108,0.902045,0.302361,0.61531,0.131507,0.225393,0.0508019
XGBoost_3_AutoML_3_20220629_183108,0.901969,0.194533,0.587154,0.128295,0.231816,0.0537385
XRT_1_AutoML_3_20220629_183108,0.901799,0.260677,0.624442,0.104895,0.225108,0.0506736


In [14]:
# пересоздаем итоговый конвейер

# создаем конвейер для количественных переменных
num_pipe = Pipeline([
    ('imputer', SimpleImputer())
])

# создаем конвейер для категориальных переменных
cat_pipe = Pipeline([
    ('imputer', SimpleImputer())
])

# создаем список трехэлементных кортежей, в котором
# первый элемент кортежа - название конвейера с
# преобразованиями для определенного типа признаков
transformers = [('num', num_pipe, num_columns),
                ('cat', cat_pipe, cat_columns)]

# создаем конвейер
pipe = Pipeline([
    ('tr', ColumnTransformer(transformers=transformers)),
    ('classifier', H2OAutoMLClassifier(preprocessing=None,
                                       max_models=10, seed=2022))])

In [15]:
# задаем сетку гиперпараметров
hyperparams_dct = {
    'tr__num__imputer__strategy': ['mean', 'median', 'constant'],
    'tr__cat__imputer__strategy': ['most_frequent', 'constant']
}
# запускаем поиск по сетке
grid = GridSearchCV(pipe, cv=3, param_grid=hyperparams_dct)
grid.fit(X_train, y_train);

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |█████████████████████████████████

In [16]:
# получаем прогнозы для тестовой выборки
predictions = grid.predict(X_test)
# оценим правильность на тестовой выборке
acc = accuracy_score(y_test, predictions)
print("Правильность на тестовой выборке: %.3f" % acc)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%
Правильность на тестовой выборке: 0.941


In [17]:
# еще можно так

# извлекаем наилучшую модель
best = grid.best_estimator_
# получаем прогнозы для тестовой выборки
predictions = best.predict(X_test)
# оценим правильность на тестовой выборке
acc = accuracy_score(y_test, predictions)
print("Правильность на тестовой выборке: %.3f" % acc)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%
Правильность на тестовой выборке: 0.941


In [18]:
# смотрим наилучшие значения гиперпараметров
print("Наилучшие значения гиперпараметров:\n{}".format(
    grid.best_params_))

Наилучшие значения гиперпараметров:
{'tr__cat__imputer__strategy': 'constant', 'tr__num__imputer__strategy': 'constant'}


In [19]:
# запишем результаты поиска в DataFrame
results = pd.DataFrame(grid.cv_results_)
# превращаем в сводную таблицу
table = results.pivot_table(
    values=['mean_test_score'],    
    index=['param_tr__cat__imputer__strategy',
           'param_tr__num__imputer__strategy'])
table

mean_test_score
param_tr__cat__imputer__strategy param_tr__num__imputer__strategy                 
constant                         constant                                 0.933678
                                 mean                                     0.929716
                                 median                                   0.932300
most_frequent                    constant                                 0.931611
                                 mean                                     0.932300
                                 median                                   0.932300

In [20]:
# выводим результаты AutoML
best = grid.best_estimator_['classifier']
automl = best.estimator
automl.leaderboard

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
StackedEnsemble_BestOfFamily_1_AutoML_22_20220629_185309,0.912305,0.169704,0.632853,0.0920009,0.217249,0.0471972
GBM_4_AutoML_22_20220629_185309,0.911861,0.172413,0.630323,0.0917137,0.21719,0.0471715
StackedEnsemble_AllModels_1_AutoML_22_20220629_185309,0.910989,0.170257,0.629147,0.0926685,0.217529,0.0473187
XGBoost_2_AutoML_22_20220629_185309,0.910851,0.185772,0.625292,0.122281,0.227859,0.0519196
GBM_3_AutoML_22_20220629_185309,0.910332,0.175707,0.629146,0.0921948,0.219893,0.0483529
GBM_1_AutoML_22_20220629_185309,0.909179,0.18993,0.597023,0.144038,0.23173,0.0536986
GBM_2_AutoML_22_20220629_185309,0.908555,0.179801,0.621628,0.115005,0.223249,0.0498399
XGBoost_3_AutoML_22_20220629_185309,0.902631,0.193709,0.587397,0.140597,0.23112,0.0534165
XRT_1_AutoML_22_20220629_185309,0.898481,0.23301,0.623064,0.105565,0.22602,0.051085
DRF_1_AutoML_22_20220629_185309,0.898156,0.387124,0.617498,0.116338,0.225769,0.0509716


## `H2OAutoMLRegressor` пример

In [21]:
# загружаем данные для задачи регрессии
data = pd.read_csv('Data/Flats_missing.csv', 
                   sep=';', decimal=',')
data.head()

,Rooms_Number,District,Stor,Storeys,Space_Total,Space_Living,Space_Kitchen,Balcon_Num,Lodgee_Num,lat,Long,Cost_KV
0,1,Заельцовский,13,17.0,54.1,18.0,21.2,0.0,1,55.0725,82.9069,50831.79298
1,1,Заельцовский,10,17.0,54.5,18.0,21.1,0.0,1,55.0725,82.9069,52000.00000
2,1,Центральный,8,17.0,37.0,0.0,0.0,0.0,0,55.0725,82.9068,87837.83784
3,1,Центральный,2,17.0,42.0,0.0,0.0,0.0,0,55.0725,82.9068,90238.09524
4,1,Центральный,13,17.0,28.0,0.0,0.0,0.0,0,55.0725,82.9068,110714.28570


In [22]:
# разбиваем данные на обучающие и тестовые: получаем обучающий
# массив признаков, тестовый массив признаков, обучающий массив
# меток, тестовый массив меток
X_train, X_test, y_train, y_test = train_test_split(
    data.drop('Cost_KV', axis=1), 
    data['Cost_KV'],
    test_size=0.3,
    random_state=42)

X_train = X_train.values
X_test = X_test.values

y_train = y_train.values
y_test = y_test.values

# создаем конвейер
pipe = Pipeline([
    ('imp', SimpleImputer(strategy='constant')),
    ('regressor', H2OAutoMLRegressor(preprocessing=None, 
                                     max_models=10, seed=2022))
])

# запускаем AutoML
pipe.fit(X_train, y_train);

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%


In [23]:
# получаем прогнозы для тестовой выборки
predictions = pipe.predict(X_test)
# оцениваем RMSE на тестовой выборке
rmse = mean_squared_error(y_test, predictions, squared=False)
print("RMSE на тестовой выборке: %.3f" % rmse)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%
RMSE на тестовой выборке: 9483.515


In [24]:
# выводим результаты AutoML
automl = pipe.named_steps.regressor.estimator
automl.leaderboard

model_id,rmse,mse,mae,rmsle,mean_residual_deviance
StackedEnsemble_AllModels_1_AutoML_23_20220629_185527,11109.4,1.23418e+08,6734.8,0.181944,1.23418e+08
StackedEnsemble_BestOfFamily_1_AutoML_23_20220629_185527,11160.9,1.24565e+08,6771.15,0.182826,1.24565e+08
GBM_4_AutoML_23_20220629_185527,11227.1,1.26049e+08,6851.11,0.184222,1.26049e+08
GBM_1_AutoML_23_20220629_185527,11304.2,1.27785e+08,6924,0.184864,1.27785e+08
GBM_3_AutoML_23_20220629_185527,11348.7,1.28793e+08,6961.28,0.186158,1.28793e+08
DRF_1_AutoML_23_20220629_185527,11358.3,1.2901e+08,6921.17,0.185917,1.2901e+08
XRT_1_AutoML_23_20220629_185527,11360.2,1.29053e+08,6908.06,0.185547,1.29053e+08
GBM_2_AutoML_23_20220629_185527,11395,1.29846e+08,7012.8,0.186889,1.29846e+08
XGBoost_3_AutoML_23_20220629_185527,11622.4,1.35079e+08,7163.02,0.190612,1.35079e+08
XGBoost_2_AutoML_23_20220629_185527,11628.4,1.3522e+08,7110.53,0.190115,1.3522e+08


In [25]:
# извлекаем нужную нам модель
h2o.get_model('DRF_1_AutoML_23_20220629_185527')

Model Details
H2ORandomForestEstimator :  Distributed Random Forest
Model Key:  DRF_1_AutoML_23_20220629_185527


Model Summary: 


,,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
0,,47.0,47.0,9672250.0,20.0,20.0,20.0,14165.0,19153.0,16379.915




ModelMetricsRegression: drf
** Reported on train data. **

MSE: 133451062.30450475
RMSE: 11552.102072978092
MAE: 7012.203089158242
RMSLE: 0.1881363850778693
Mean Residual Deviance: 133451062.30450475

ModelMetricsRegression: drf
** Reported on cross-validation data. **

MSE: 129010212.29656787
RMSE: 11358.266253991755
MAE: 6921.1746908948635
RMSLE: 0.18591658290574456
Mean Residual Deviance: 129010212.29656787

Cross-Validation Metrics Summary: 


,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,mae,6.921175e+03,1.052039e+02,7.100097e+03,6.859382e+03,6.833760e+03,6.892952e+03,6.919682e+03
1,mean_residual_deviance,1.290102e+08,3.726303e+07,1.835246e+08,9.556202e+07,9.237683e+07,1.363714e+08,1.372162e+08
2,mse,1.290102e+08,3.726303e+07,1.835246e+08,9.556202e+07,9.237683e+07,1.363714e+08,1.372162e+08
3,r2,4.918372e-01,6.632507e-02,4.079469e-01,5.590534e-01,5.598539e-01,4.730992e-01,4.592325e-01
4,residual_deviance,1.290102e+08,3.726303e+07,1.835246e+08,9.556202e+07,9.237683e+07,1.363714e+08,1.372162e+08
5,rmse,1.126515e+04,1.622744e+03,1.354713e+04,9.775583e+03,9.611287e+03,1.167782e+04,1.171393e+04
6,rmsle,1.858941e-01,3.230138e-03,1.908681e-01,1.870980e-01,1.828036e-01,1.835666e-01,1.851344e-01



Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_mae,training_deviance
0,,2022-06-29 18:58:49,1 min 9.987 sec,0.0,NaN,NaN,NaN
1,,2022-06-29 18:58:51,1 min 11.211 sec,5.0,13695.991626,8473.332086,1.875802e+08
2,,2022-06-29 18:58:52,1 min 12.500 sec,10.0,12417.934281,7771.040961,1.542051e+08
3,,2022-06-29 18:58:53,1 min 13.760 sec,15.0,12020.315361,7429.648955,1.444880e+08
4,,2022-06-29 18:58:55,1 min 15.183 sec,20.0,11824.621057,7267.979248,1.398217e+08
5,,2022-06-29 18:58:56,1 min 16.374 sec,25.0,11778.795829,7192.458548,1.387400e+08
6,,2022-06-29 18:58:57,1 min 17.619 sec,30.0,11693.074419,7134.751883,1.367280e+08
7,,2022-06-29 18:58:58,1 min 18.862 sec,35.0,11671.958821,7092.658614,1.362346e+08
8,,2022-06-29 18:59:00,1 min 20.172 sec,40.0,11631.391712,7050.513680,1.352893e+08
9,,2022-06-29 18:59:01,1 min 21.409 sec,45.0,11572.892490,7023.526991,1.339318e+08



Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,C10,6.105234e+13,1.000000,0.178531
1,C5,5.017526e+13,0.821840,0.146724
2,C4,4.439773e+13,0.727208,0.129829
3,C11,4.431821e+13,0.725905,0.129597
4,C2,4.262079e+13,0.698102,0.124633
5,C6,2.853812e+13,0.467437,0.083452
6,C3,2.279577e+13,0.373381,0.066660
7,C7,2.279501e+13,0.373368,0.066658
8,C1,1.273342e+13,0.208566,0.037236
9,C9,6.796561e+12,0.111324,0.019875


In [26]:
# извлекаем наилучшую модель из семейства GBM
gbm = automl.get_best_model(algorithm='gbm')
gbm

Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Machine
Model Key:  GBM_4_AutoML_23_20220629_185527


Model Summary: 


,,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
0,,76.0,76.0,247659.0,10.0,10.0,10.0,28.0,570.0,254.92105




ModelMetricsRegression: gbm
** Reported on train data. **

MSE: 91071507.0223175
RMSE: 9543.139264535414
MAE: 6043.618145537464
RMSLE: 0.1634354774450186
Mean Residual Deviance: 91071507.0223175

ModelMetricsRegression: gbm
** Reported on cross-validation data. **

MSE: 126048644.99227272
RMSE: 11227.138771399983
MAE: 6851.108989652716
RMSLE: 0.18422174508207162
Mean Residual Deviance: 126048644.99227272

Cross-Validation Metrics Summary: 


,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,mae,6.852157e+03,7.798803e+01,6.987180e+03,6.842608e+03,6.795463e+03,6.803161e+03,6.832372e+03
1,mean_residual_deviance,1.261204e+08,3.547153e+07,1.779565e+08,9.448508e+07,9.100948e+07,1.327993e+08,1.343517e+08
2,mse,1.261204e+08,3.547153e+07,1.779565e+08,9.448508e+07,9.100948e+07,1.327993e+08,1.343517e+08
3,r2,5.027447e-01,6.122913e-02,4.259097e-01,5.640227e-01,5.663689e-01,4.869006e-01,4.705215e-01
4,residual_deviance,1.261204e+08,3.547153e+07,1.779565e+08,9.448508e+07,9.100948e+07,1.327993e+08,1.343517e+08
5,rmse,1.114303e+04,1.562582e+03,1.334003e+04,9.720344e+03,9.539889e+03,1.152386e+04,1.159102e+04
6,rmsle,1.842326e-01,2.544359e-03,1.874820e-01,1.860914e-01,1.816803e-01,1.819298e-01,1.839797e-01



Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_mae,training_deviance
0,,2022-06-29 19:00:34,28.043 sec,0.0,15806.495979,11372.874380,2.498453e+08
1,,2022-06-29 19:00:34,28.415 sec,5.0,13387.457468,9212.778927,1.792240e+08
2,,2022-06-29 19:00:34,28.816 sec,10.0,12029.020153,7950.570357,1.446973e+08
3,,2022-06-29 19:00:35,29.196 sec,15.0,11274.768178,7258.041721,1.271204e+08
4,,2022-06-29 19:00:35,29.569 sec,20.0,10813.847852,6880.898604,1.169393e+08
5,,2022-06-29 19:00:35,29.910 sec,25.0,10528.958380,6659.966946,1.108590e+08
6,,2022-06-29 19:00:36,30.243 sec,30.0,10327.773275,6503.082956,1.066629e+08
7,,2022-06-29 19:00:36,30.596 sec,35.0,10152.357011,6393.695494,1.030704e+08
8,,2022-06-29 19:00:36,30.932 sec,40.0,10049.883660,6325.451384,1.010002e+08
9,,2022-06-29 19:00:37,31.272 sec,45.0,9978.449029,6292.059113,9.956945e+07



Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,C2,6.972071e+12,1.000000,0.187413
1,C10,6.815666e+12,0.977567,0.183209
2,C11,6.032062e+12,0.865175,0.162145
3,C5,5.251109e+12,0.753163,0.141153
4,C4,4.944036e+12,0.709120,0.132899
5,C6,2.110102e+12,0.302651,0.056721
6,C7,1.760673e+12,0.252532,0.047328
7,C3,1.519134e+12,0.217888,0.040835
8,C1,8.789295e+11,0.126064,0.023626
9,C9,5.931042e+11,0.085069,0.015943


In [27]:
# смотрим параметры и гиперпараметры извлеченной модели
gbm.params

{'auc_type': {'actual': 'AUTO', 'default': 'AUTO', 'input': 'AUTO'},
 'balance_classes': {'actual': False, 'default': False, 'input': False},
 'build_tree_one_node': {'actual': False, 'default': False, 'input': False},
 'calibrate_model': {'actual': False, 'default': False, 'input': False},
 'calibration_frame': {'actual': None, 'default': None, 'input': None},
 'categorical_encoding': {'actual': 'Enum',
  'default': 'AUTO',
  'input': 'AUTO'},
 'check_constant_response': {'actual': True, 'default': True, 'input': True},
 'checkpoint': {'actual': None, 'default': None, 'input': None},
 'class_sampling_factors': {'actual': None, 'default': None, 'input': None},
 'col_sample_rate': {'actual': 0.8, 'default': 1.0, 'input': 0.8},
 'col_sample_rate_change_per_level': {'actual': 1.0,
  'default': 1.0,
  'input': 1.0},
 'col_sample_rate_per_tree': {'actual': 0.8, 'default': 1.0, 'input': 0.8},
 'custom_distribution_func': {'actual': None, 'default': None, 'input': None},
 'custom_metric_func'